In [1]:
from sub_func import *

/opt/anaconda3/envs/finTF/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
pf_selection_report_format = """{
report: '(생성한 레포트 본문 전체)',
sector: '[레포트 내에 언급된 주요 섹터를 리스트 형식으로 반환]'
}
"""

pf_selection_system = f"""당신은 증권회사의 포트폴리오 전문가입니다.
주어진 다음 정보들은 이전 분기의 국제 뉴스, 거시경제 정보, 인덱스 지표들의 가격입니다.
이를 바탕으로 다음 분기에 주목해야 할 섹터를 정리해서 보고서 형식으로 출력해야 합니다.
보고서의 의견은 반드시 주어진 정보에 기반하여야 합니다.
보고서의 형식은 반드시 다음을 따라야 합니다. {pf_selection_report_format}

** Return only pure JSON format without any code block or delimiters. **
** Make sure that the response does not create JSON decode error. **
"""

pf_selection_prompt = f""""""

# ================================ #

In [3]:
target_year = '2022'
target_quarter = 'Q4'

start_date = '20221001'
end_date = '20221231'

In [4]:
# ==== 국제 뉴스 수집 ==== #
intl_news_title_list = list(intl_news_info(target_year, start_date, end_date)['news_title'])
pf_selection_prompt += f"""국제 뉴스 헤드라인: {intl_news_title_list}\n"""

# ==== 거시경제 정보 수집 ==== #
def create_macro_econ_dict(country, econ_item):
    result_dict = macro_econ_dict[country][econ_item].set_index('Date').to_dict('index')
    final_dict = {k: v[list(v.keys())[0]] for k, v in result_dict.items()}

    return final_dict

reports_dict = {}
macro_econ_dict = macro_econ_info(target_year, start_date, end_date)
country_list = list(macro_econ_dict.keys())

for country in country_list:
    target_country_dict = macro_econ_dict[country]
    econ_items = list(target_country_dict.keys())
    for econ_item in econ_items:
        final_dict = create_macro_econ_dict(country, econ_item)
        reports_dict[f"{country}_{econ_item}"] = final_dict
pf_selection_prompt += f"""국가별 거시경제 정보: {reports_dict}\n"""

# ==== 인덱스 가격 수집 ==== #
index_prices = {}
sector_list = [s for s in os.listdir('../store_data/raw/market_data/sector') if '코스피' not in s]

for sector in sector_list:
    # 토큰수 제한때문에 컬럼 선별해서 넣기...
    index_price = index_price_info(sector, start_date, end_date)[['Close', 'Transaction_Val','Market_Cap', 'RSI_14']]
    index_prices[sector] = index_price.T.to_dict()

pf_selection_prompt += f"""인덱스 가격 지표: {index_prices}\n"""

UK에 대해 Inflation Rate 정보를 찾을 수 없습니다. | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/FRED/UK/Inflation Rate/2022/2022_Inflation Rate.csv'


In [5]:
stock_selection_response = to_GPT(pf_selection_system, pf_selection_prompt)

# 여기까지 해서, 투자할 섹터 결정
try:
    sample = eval(stock_selection_response['choices'][0]['message']['content'])
    pf_selection_report = sample['report']
    pf_selection_sector = sample['sector']
except Exception as e:
    print('-'*50)
    print(f"GPT 응답 구조가 올바르지 않음: {e}")

In [6]:
pf_selection_sector

['금융업', '반도체', '화학', '기계', '유통업', '서비스업']

In [7]:
temp_pf = get_pf(target_year, target_quarter, pf_selection_sector)

포트폴리오 구성 시작...
--------------------------------------------------
(1/792)
61439
--------------------------------------------------
(2/792)
83073
--------------------------------------------------
(3/792)
14735
--------------------------------------------------
(4/792)
69710
--------------------------------------------------
(5/792)
23730
--------------------------------------------------
(6/792)
25607
--------------------------------------------------
(7/792)
62002
--------------------------------------------------
(8/792)
35300
--------------------------------------------------
(9/792)
42312
--------------------------------------------------
(10/792)
35622
--------------------------------------------------
(11/792)
16648
--------------------------------------------------
(12/792)
23223
--------------------------------------------------
(13/792)
22656
--------------------------------------------------
(14/792)
8661
--------------------------------------------------
(15/792)
027390에 해당하

In [8]:
temp_pf

{'금융업': {'033660': 0.2874687311337993,
  '035720': 0.2786517823316005,
  '020000': 0.2786017954349518,
  '031430': 0.27843669056892395,
  '093050': 0.27675442553386226,
  '000480': 0.2762377858161926,
  '004370': 0.2758499085903168,
  '000080': 0.2752651572227478,
  '011420': 0.2739572823047638,
  '001740': 0.27374470233917236},
 '반도체': {'042700': 0.3219891715132768,
  '195870': 0.31157045236552255,
  '077500': 0.2901709899198932,
  '000990': 0.2540645369630353,
  '003160': 0.2475533400134503,
  '029460': 0.2460166213332186,
  '007810': 0.23669428200179993,
  '092220': 0.22622363511103638,
  '025560': 0.2214982170068136,
  '000670': 0.22115335409299894},
 '화학': {'003720': 0.3262245132366421,
  '025860': 0.29475844552991526,
  '014680': 0.294188588119612,
  '011690': 0.28395650695400604,
  '002840': 0.2834003232634898,
  '069260': 0.2766124121978539,
  '006060': 0.27300674413942794,
  '004430': 0.27130205828153653,
  '024070': 0.2692303115405198,
  '012610': 0.2679426830820179},
 '기계': 

In [9]:
temp_tickers_list = []

for sector in temp_pf:
    for ticker in temp_pf[sector]:
        if ticker not in temp_tickers_list:
            temp_tickers_list.append(ticker)

In [10]:
store_all_raw_pf_selection = {}

In [11]:
pf_selection_corp_format = """{
report: '(생성한 레포트 본문 전체)',
invest: '(True or False로만 답하세요.)'
}
"""

pf_selection_corp_system = f"""당신은 증권회사의 분석가입니다.
주어진 자료들을 바탕으로 다음 분기동안 해당 종목을 보유해야할지 판단하고 이를 보고서 형식으로 작성해야 합니다.
보고서의 의견은 반드시 주어진 정보에 기반하여야 합니다.
보고서의 형식은 반드시 다음을 따라야 합니다. {pf_selection_corp_format}

** Return only pure JSON format without any code block or delimiters. **
** Make sure that the response does not create JSON decode error. **
"""

pf_selection_corp_prompt = f""""""

In [12]:
for ticker in temp_tickers_list:
    pf_selection_corp_prompt = ""
    corp_rel_news_df = None
    SA_result_corp_rel_news_dict = None
    price_dict = None
    fin_statement_df = None
    fin_report_df = None

    try:
        corp_rel_news_df = corp_rel_news_info(ticker, target_year, start_date, end_date)
        corp_rel_news_df = corp_rel_news_df[corp_rel_news_df['news_category'].str.contains('증권')]

        # 활용할 최종 정보!
        SA_result_corp_rel_news_dict = {
            'positive': [],
            'negative': []
        }

        for i in range(len(corp_rel_news_df)):
            news_title = corp_rel_news_df.iloc[i]['news_title']
            SA_result = get_SA_result(news_title)

            if SA_result['label'] == 'positive' and SA_result['prob'] > 0.9:
                SA_result_corp_rel_news_dict['positive'].append(news_title)
            elif SA_result['label'] == 'negative' and SA_result['prob'] > 0.9:
                SA_result_corp_rel_news_dict['negative'].append(news_title)

        pf_selection_corp_prompt += f"기업 관련 뉴스: {SA_result_corp_rel_news_dict}"

    except:
        print(f"{ticker}에 대해 뉴스를 찾을 수 없습니다.")

    try:
        price_dict = stock_price_info(ticker, start_date, end_date)[['Close', 'MA_20', 'RSI_14']].T.to_dict()
        pf_selection_corp_prompt += f"가격 정보: {price_dict}"
        
    except:
        print(f"{ticker}에 대해 가격 정보를 찾을 수 없습니다.")

    try:
        fin_statement_df = fin_statement_info(ticker, target_year, target_quarter)
        pf_selection_corp_prompt += f"재무비율: {fin_statement_df}"
    except:
        print(f"{ticker}에 대해 재무 비율을 찾을 수 없습니다..")

    try:
        main_reports_df = reports_info(ticker, target_year, target_quarter)
        fin_report_df = main_reports_df['1. 요약재무정보.csv'][0]
        pf_selection_corp_prompt += f"재무보고서: {fin_report_df}"
    except:
        print(f"{ticker}에 대해 재무보고서를 찾을 수 없습니다..")

    pf_selection_corp_response = to_GPT(pf_selection_corp_system, pf_selection_corp_prompt)
    try:
        final_pf_selection_corp_response = eval(pf_selection_corp_response['choices'][0]['message']['content'])
        store_all_raw_pf_selection[ticker] = final_pf_selection_corp_response
    except Exception as e:
        print(f"{ticker}에 대한 분석의 GPT 응답 구조에 오류가 있습니다. | {e}")

033660에 대해 뉴스를 찾을 수 없습니다.
/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/market_data/price/033660/2022.10/2022.10_033660.csv 파일을 찾을 수 없습니다.
/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/market_data/price/033660/2022.11/2022.11_033660.csv 파일을 찾을 수 없습니다.
/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/market_data/price/033660/2022.12/2022.12_033660.csv 파일을 찾을 수 없습니다.
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/033660/_033660_재무제표 ().csv'
재무제표를 불러오는 과정에서 오류가 발생했습니다 | [Errno 2] No such file or directory: '/Users/yeonsuk/investment/finTF/pipeline/sub_func/get_info/../../../store_data/raw/opendart/store_financial_statement/033660/_033660_재무제표 ().csv'
033660의 이전 년도 재무 데이터를 불러올 수 없습니다.
033660의 fin_statement_info 정보를 확인할 수 없습니다.
재무제표를 불러오는 과정에서 오류가 발

In [13]:
store_all_raw_pf_selection

{'033660': {'report': '해당 종목의 최근 분기 재무제표를 분석한 결과, 자산총계는 12,581,473 백만원으로 증가하였으며, 영업수익 또한 1,324,198 백만원으로 증가세를 보이고 있습니다. 영업이익은 253,214 백만원으로 전년 대비 상승하였으며, 당기순이익은 183,328 백만원으로 긍정적인 성장세를 나타내고 있습니다. 그러나, 차입부채는 10,452,578 백만원으로 상당히 높은 수준이며, 총부채는 11,040,754 백만원에 달합니다. 이는 재무적인 부담을 증가시킬 수 있는 요소입니다. 또한, 이익잉여금은 증가하고 있으나, 자본총계는 1,540,719 백만원으로 여전히 자본조정이 필요한 상황입니다. 종합적으로 볼 때, 해당 종목은 영업수익과 이익이 증가하고 있지만, 높은 차입부채와 부채총계는 리스크 요소로 작용할 수 있습니다. 따라서, 보유 여부에 대한 판단은 신중해야 하며, 지속적인 모니터링이 필요합니다.',
  'invest': 'False'},
 '035720': {'report': '카카오는 최근 몇 가지 긍정적인 소식과 부정적인 소식이 혼재하고 있는 상황입니다. 긍정적인 면에서는 목표가 조정에도 불구하고 하반기 콘텐츠 성과가 주가 반등을 이끌 것이라는 기대가 있으며, 외국인과 기관의 매수세가 강하게 나타나고 있습니다. 특히 카카오 4형제의 주가는 최근 몇 차례 급등하며 체결 강도가 강세를 보이고 있습니다. 반면, 부정적인 정보로는 카카오의 주가가 여러 차례 신저가를 기록하고 있으며, 문어발식 자회사 상장 논란과 데이터센터 화재로 인한 신뢰성 훼손 등의 이슈가 존재합니다. 이러한 부정적 요소들은 주가에 상당한 압박을 주고 있으며, 성장성 둔화 우려도 커지고 있습니다. 최근 재무정보를 보면 카카오는 매출과 영업이익에서 긍정적인 성장을 보였지만, 당기순이익에서는 큰 변동성이 있었습니다. 종합적으로 볼 때, 카카오는 현재 긍정적인 모멘텀과 부정적인 리스크가 공존하는 상황으로, 향후 분기 동안 주가의 변동성이 클 것으로 예상됩니

In [16]:
# invest가 'True'인 항목만 필터링하는 코드
filtered_temp_pf = {
    sector: {ticker: value for ticker, value in tickers.items()
             if store_all_raw_pf_selection.get(ticker, {}).get('invest') == 'True'}
    for sector, tickers in temp_pf.items()
}

filtered_temp_pf

{'금융업': {'035720': 0.2786517823316005,
  '020000': 0.2786017954349518,
  '031430': 0.27843669056892395,
  '093050': 0.27675442553386226,
  '004370': 0.2758499085903168,
  '011420': 0.2739572823047638,
  '001740': 0.27374470233917236},
 '반도체': {'077500': 0.2901709899198932,
  '000990': 0.2540645369630353,
  '003160': 0.2475533400134503,
  '029460': 0.2460166213332186,
  '007810': 0.23669428200179993,
  '092220': 0.22622363511103638,
  '025560': 0.2214982170068136},
 '화학': {'003720': 0.3262245132366421,
  '025860': 0.29475844552991526,
  '014680': 0.294188588119612,
  '069260': 0.2766124121978539,
  '006060': 0.27300674413942794,
  '004430': 0.27130205828153653},
 '기계': {'267270': 0.2348374845710213,
  '042670': 0.21426483828068155,
  '241560': 0.19270413366487774,
  '010100': 0.17711771436909235,
  '010400': 0.16121084763353144,
  '003650': 0.15260156045548973,
  '002900': 0.1493362733196124,
  '267250': 0.1491620849880163},
 '유통업': {'037710': 0.2724596243649255,
  '282330': 0.260356872

In [15]:
final_pf_list = []

for ticker in store_all_raw_pf_selection:
    if store_all_raw_pf_selection[ticker]['invest'] == 'True':
        final_pf_list.append(ticker)

In [16]:
final_pf_list

['033660',
 '020000',
 '031430',
 '093050',
 '000480',
 '004370',
 '011420',
 '001740',
 '180640',
 '009180',
 '003490',
 '001140',
 '074610',
 '003300',
 '025860',
 '014680',
 '002840',
 '069260',
 '006060',
 '004430',
 '024070',
 '102460',
 '214390',
 '207940',
 '128940',
 '017180',
 '293480',
 '063160',
 '007460',
 '001360']

In [18]:
from pykrx import stock
import pandas as pd

def calculate_return(ticker, start_date, end_date):
    """
    Calculate the return for a given ticker between start_date and end_date.

    Parameters:
    ticker (str): Stock ticker
    start_date (str): Start date in 'YYYYMMDD' format
    end_date (str): End date in 'YYYYMMDD' format

    Returns:
    float: Return in percentage between start_date and end_date
    """
    try:
        # Get OHLCV data
        df = stock.get_market_ohlcv(start_date, end_date, ticker)
        
        # Ensure data is available
        if df.empty:
            return f"No data available for ticker {ticker} between {start_date} and {end_date}."
        
        # Calculate return
        start_price = df.iloc[0]['종가']
        end_price = df.iloc[-1]['종가']
        
        # Return in percentage
        return_percentage = ((end_price - start_price) / start_price) * 100
        return return_percentage
    except Exception as e:
        return f"An error occurred: {e}"

In [19]:
start_date = "20230101"
end_date = "20230331"

# Example usage
for ticker in final_pf_list:
    result = calculate_return(ticker, start_date, end_date)
    if isinstance(result, float):
        print(f"The return for ticker {ticker} from {start_date} to {end_date} is {result:.2f}%.")
    else:
        print(result)

No data available for ticker 033660 between 20230101 and 20230331.
The return for ticker 020000 from 20230101 to 20230331 is 0.19%.
The return for ticker 031430 from 20230101 to 20230331 is -10.72%.
The return for ticker 093050 from 20230101 to 20230331 is 10.51%.
The return for ticker 000480 from 20230101 to 20230331 is 21.77%.
The return for ticker 004370 from 20230101 to 20230331 is 7.67%.
The return for ticker 011420 from 20230101 to 20230331 is 53.31%.
The return for ticker 001740 from 20230101 to 20230331 is 17.78%.
The return for ticker 180640 from 20230101 to 20230331 is 28.71%.
The return for ticker 009180 from 20230101 to 20230331 is 4.04%.
The return for ticker 003490 from 20230101 to 20230331 is 2.20%.
The return for ticker 001140 from 20230101 to 20230331 is -14.71%.
The return for ticker 074610 from 20230101 to 20230331 is 120.27%.
The return for ticker 003300 from 20230101 to 20230331 is 5.44%.
The return for ticker 025860 from 20230101 to 20230331 is 9.17%.
The return f

In [25]:
0.19-10.72+10.51+21.77+7.67+53.31+17.78+28.71+4.04+2.20-14.71+120.27+5.44+9.17+28.76+2.96+14.14+1.87+8.84+4.80+18.31-36.27-4.72-8.41+-18.12-16.46+4.61-13.31-2.42

240.20999999999992

30